In [1]:


import requests
from bs4 import BeautifulSoup
import csv
import os
import itertools
import time
import random
import urllib.robotparser

#Revisant robots.txt es veu sobre quines pagines no es pot fer scrapping. Es crea una funció per a comprobar si es permet
def robot(url_robot, url):
    rp = urllib.robotparser.RobotFileParser()
    rp.set_url(url_robot)
    rp.read()
    user_agent = 'wswp'
    return print('Available:', rp.can_fetch(user_agent, url))

url = 'https://www.amazon.es/gp/bestsellers/books?pf_rd_r=3SGMV0M1FJ0N0CB7MQC0&pf_rd_p=d3259a28-7b77-56ff-ade0-8a02db82407a&pd_rd_r=0c54ad8c-b4d5-4d1a-a3bb-3246e048bdc2&pd_rd_w=m1O3G&pd_rd_wg=ES5Zj&ref_=pd_gw_ri'
url_robot = 'http://www.amazon.com/robots.txt'

robot(url_robot, url)

#S'utilitzen certs parametres de headers per a que el host no detecti que no s'esta utilizant un buscador

headers = {
    'authority': 'www.amazon.es',
    'accept': 'text/html,*/*',
    'sec-fetch-dest': 'empty',
    'x-requested-with': 'XMLHttpRequest',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36',
    'content-type': 'application/x-www-form-urlencoded',
    'origin': 'https://www.amazon.es',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'referer': 'https://www.amazon.es/Aprendiendo-repasar-actividades-infantiles-preescolar/dp/164608070X/ref=zg_bs_books_1?_encoding=UTF8&psc=1&refRID=56ZR35YMVJ6JCAVDV35P',
    'accept-language': 'es-419,es;q=0.9,ca;q=0.8,en;q=0.7',
    'cookie': 'session-id=261-6732941-4022622; i18n-prefs=EUR; csm-hit=tb:s-70XB9ADC3KQKXZNWJ0H7|1586340018542&t:1586340018543; ubid-acbes=261-5214057-4062537; session-id-time=2082758401l; x-wl-uid=1496UkvL3TN/WMSFXNQFQ1TustQRy4vYymz7YCTDp4ebwWWg/9S+j8ZRC8pSw4QG3oYYfusYOZeU=; session-token=wLXXRXA0YpGTaAacKe1aHCqbepbrHvK6d2g816+i0fxdM35bmBFcV0u8dGiDyDasRP9SlGhyBNmO2x8VLgBdBnMFCqOUPYkF5RBy8HBlFkgdFFLyZYcaNJATcd2g36Zac81CZyM0qXovTLe0gc+49/jJdCkOCBDkPft5kXi4lmgAvpWOPSY1Z4hCM30Q2AvbyNhhTOKQ7bOfMsxeAAS4j6QvKruki1g51APumKIwkCJ9J9W21CIu14EyuheLO2Pv',
}

#Es crea una funció per detectar si l'atribut no existeix. En cas de que no existeixi es deixara com una cadena buida: 
def trobar_valor(identificador):
    
    try:
        caracteristica = descripcio.find(class_= identificador).string
    except:
        caracteristica = ""
    return caracteristica

def scrapp(url, headers = headers):
    
    page = requests.get(url)
    soup = BeautifulSoup(page.content)
    return soup
    
        
#Es declara la variable on s'enmagatzemaran els resultats

llista_llibres = []
dictionari_atribut = {"Nom": "",
                      "Autor": "",
                      "Puntuacio": "",
                      "Num_resenyes": "",
                      "Categoria": "",
                      "Preu": "",
                      'Editor': "",
                      'Idioma': "",
                      'ISBN-10': "",
                      'ISBN-13': "",
                      'Coleccion': "",
                      'Edad recomendada': ""}

#Es crea l'scrapp
#El llistat de llibres de Amazon es divideix en dues pagines, pel que hem de llegir aquestes dues.
for pagina in range(1,3):
    url = "https://www.amazon.es/gp/bestsellers/books/ref=zg_bs_pg_1/259-0380097-5900722?ie=UTF8&pg=%d" % pagina
    soup = scrapp(url)

    # Guardem en la variable llistat el contingut de tots els llibres
    llistat = soup.find(class_="a-ordered-list a-vertical")

    # Recorrem llibre a llibre (aok-inline-block zg-item) buscant les característiques de cada un d'ells. 

    for descripcio in llistat.find_all('span', class_="aok-inline-block zg-item"):
        
        dictionari_atribut["Nom"] = trobar_valor("p13n-sc-truncate").strip()
        dictionari_atribut["Autor"] = trobar_valor("a-row a-size-small")
        dictionari_atribut["Puntuacio"] = trobar_valor("a-icon-alt")
        dictionari_atribut["Num_resenyes"] = trobar_valor("a-size-small a-link-normal")
        dictionari_atribut["Categoria"] = trobar_valor("a-size-small a-color-secondary")
        dictionari_atribut["Preu"] = trobar_valor("p13n-sc-price")
        
        #Dintre de cada llibre s'accedeix a la pagina del prodcute per a obtenir més informació
        link = descripcio.find(class_="a-link-normal").get('href')
        link = "https://www.amazon.es" + str(link)
        print(link)
        time.sleep(random.random())
        page2 = requests.get(link, headers = headers)
        soup2 = BeautifulSoup(page2.content)
        
        #Es selecciona la taula d'on es vol treure informació
        taula = soup2.find(id ="detail_bullets_id")
        for element in taula.find_all('li'):
            text = element.get_text()
            posicio_punts = text.find(":")
            if text[:posicio_punts] in ['Editor', 'Idioma', 'ISBN-10', 'ISBN-13', 'Coleccion', 'Edad recomendada']:
                 dictionari_atribut[text[:posicio_punts]] = text[posicio_punts + 2:]
            elif text[:4] == 'Tapa':
                dictionari_atribut['Paginas'] = text[posicio_punts + 2:]
                
        #Copiem les característiques de cada llibre en una línea del CSV final, utilitzant ";" com a separador
        #print(dictionari_atribut)
        linea = list(dictionari_atribut.values())
        linea = ";".join(linea)
        llista_llibres.append(linea)
        
        #Es buida el diccionari per si hi ha algun atribut que no tingui cap valor en la seguent iteració
        dictionari_atribut = dictionari_atribut.fromkeys(dictionari_atribut, "")
        
                
# Escrivim un fitxer csv amb les dades dels llibres més venuts d'amazon
with open('llibres.csv', 'w', newline='') as csvFile:
    csvFile.write("Nom;Autor;Puntuacio;Num_resenyes;Categoria;Preu;Pagines;Editor;Idioma;ISBN-10,ISBN-13,Coleccion,Edad recomendada\n")
    for llibre in llista_llibres:
        csvFile.write(llibre)
        csvFile.write('\n')


Available: True
https://www.amazon.es/Aprendiendo-repasar-actividades-infantiles-preescolar/dp/164608070X?_encoding=UTF8&psc=1
https://www.amazon.es/Libro-colorear-para-adultos-Pensamientos/dp/1729502148?_encoding=UTF8&psc=1
https://www.amazon.es/Caligraf%C3%ADa-Escritura-Min%C3%BAsculas-Iniciaci%C3%B3n-May%C3%BAsculas/dp/8485109279?_encoding=UTF8&psc=1
https://www.amazon.es/Ejercicios-c%C3%A1lculo-a%C3%B1os-Aprendo-jugando/dp/8498257115?_encoding=UTF8&psc=1
https://www.amazon.es/Primeros-ejercicios-escritura-Aprendo-jugando/dp/8498255147?_encoding=UTF8&psc=1
https://www.amazon.es/caso-castillo-encantado-Serie-BuscaPistas/dp/8415580703?_encoding=UTF8&psc=1
https://www.amazon.es/Aprendamos-abecedario-actividades-vocabulario-comprensi%C3%B3n/dp/1646080386?_encoding=UTF8&psc=1
https://www.amazon.es/Pepo-orinal-Castellano-Proyecto-Emocionales/dp/8469604899?_encoding=UTF8&psc=1
https://www.amazon.es/Amor-Colorear-antiestr%C3%A9s-adolescentes-relajaci%C3%B3n/dp/1533575231?_encoding=UTF8&psc=

https://www.amazon.es/Los-Futbol%C3%ADsimos-El-misterio-castillo-embrujado/dp/846757769X?_encoding=UTF8&psc=1
https://www.amazon.es/zapatos-Valeria-Saga-BEST-SELLER/dp/8490628971?_encoding=UTF8&psc=1
https://www.amazon.es/Stilton-En-reino-fantas%C3%ADa-Geronimo/dp/8408060996?_encoding=UTF8&psc=1
https://www.amazon.es/TEXT-DISTRIBUCIONS-Una-historia-Espa%C3%B1a/dp/8420438170?_encoding=UTF8&psc=1
https://www.amazon.es/Diario-Greg-%C2%A1Esto-Es-Colmo/dp/8427200072?_encoding=UTF8&psc=1
https://www.amazon.es/tesoro-pirata-Aprender-MAY%C3%9ASCULA-imprenta/dp/8417210261?_encoding=UTF8&psc=1
https://www.amazon.es/LECTURAS-COMPRENSIVAS-RUBIO-S-L-U-ENRIQUE/dp/8494727567?_encoding=UTF8&psc=1
https://www.amazon.es/PREPARA-TERCERO-Repasa-contenidos-Primaria/dp/8468341312?_encoding=UTF8&psc=1
https://www.amazon.es/Sidi-HISPANICA-Arturo-P%C3%A9rez-Reverte/dp/8420435473?_encoding=UTF8&psc=1
https://www.amazon.es/Loba-negra-Trama-Juan-G%C3%B3mez-Jurado/dp/8466666494?_encoding=UTF8&psc=1
https://www.ama